In [ ]:
Remaining useful life predictions is a complex beast

In [ ]:
# Let's import everything we need

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
from crull.transformers.IonMillEtchTool import IonMillEtchTool_expanding_transformer, IonMillEtchTool_raw_transformer
import rul_pm.datasets.PHMDataset2018 as IonMillEtchToolDataset
from rul_pm.datasets.PHMDataset2018 import PHMDataset2018 as PHMDataset2018Dataset
from sklearn.model_selection import train_test_split
from temporis.iterators.utils import true_values
from crull.graphics.graphics import (histogram_categorical,
                                     add_vertical_lines_annotations)
import seaborn as sbn
from tqdm.auto import tqdm
from tensorflow.keras import Model, Input
import tensorflow_addons as tfa
from tcn import TCN
import tensorflow as tf
from tensorflow.keras.layers import (Conv1D, Dense, 
                                     Flatten, Dropout, LSTM, GlobalAveragePooling1D,
                                     Activation,
                                     LSTM, BatchNormalization, GRU, LeakyReLU,
                                     Permute,
                                    Reshape)
from tensorflow_addons.layers import WeightNormalization
from temporis.models.keras import tf_regression_dataset
import tensorflow 
sbn.set()

# Dataset description and pre-processing
Before beggining is would be to understand what is the problem we want to solver.

RUL-PM already pre-process all this for us and prepare a dataset ready to use

# Dataset loading

In [ ]:
dataset = PHMDataset2018Dataset(
    failure_types=IonMillEtchToolDataset.FailureType.FlowCoolPressureDroppedBelowLimit,
    tools='03_M02')

# EDA

# Data pre-processing

In [1]:
class ScalingPerRecipe(TransformerStep):
    def __init__(self, * , standard_scaler:bool=False):
        super().__init__(prefer_partial_fit=False)
        self.means = {}
        self.std = {}
        self.max = {}
        self.min = {}
        self.standard = standard_scaler
        self.recipes = set()


    def fit(self, X:pd.DataFrame):
        
        
        for g, data in X.groupby('recipe'):
            if self.standard:
                self.means[g] = data.mean()
                self.std[g] = data.std()
            else:
                self.max[g] = data.max()
                self.min[g] = data.min()
            self.recipes.add(g)
        return self 

    def transform(self, X:pd.DataFrame):
        X_new = X.copy().drop(columns='recipe')
        for g, data in X.groupby('recipe'):
            index = data.index
            if g not in self.recipes:
                continue
            
            if self.standard:
                X_new.loc[index, :] =(X_new.loc[index, :]-self.means[g])/self.std[g]
            else:
                X_new.loc[index, :] =(X_new.loc[index, :]-self.min[g])/(self.max[g]-self.min[g])
        return X_new

NameError: name 'TransformerStep' is not defined

## Raw transformer

In [ ]:
pipe_raw = ByNameFeatureSelector(features=features + ['recipe'])
pipe_raw = RecipeEffectRemoval()(pipe_raw)
pipe_raw = MeanImputer()(pipe_raw)

stage_cat = OneHotCategorical(feature='FIXTURESHUTTERPOSITION')

pipe_raw = Concatenate()([pipe_raw, stage_cat])


y_pipe = ByNameFeatureSelector(features=["RUL"])


raw_transformer = Transformer(
    transformerX=pipe_raw,
    transformerY=y_pipe
)


## Extracing features transformer

In [ ]:
pipe_raw = ByNameFeatureSelector(features=features + ['recipe'])
pipe_raw = RecipeEffectRemoval()(pipe_raw)
pipe_raw = MeanImputer()(pipe_raw)

expanding = ByNameFeatureSelector(features=features + ['recipe'])
expanding = RecipeEffectRemoval()(expanding)
expanding = ExpandingStatistics(to_compute=['rms', 'std', 'mean', 'std_atan'])(expanding)

stage_cat = OneHotCategorical(feature='FIXTURESHUTTERPOSITION')

pipe_raw = Concatenate()([pipe_raw, stage_cat, expanding])


y_pipe = ByNameFeatureSelector(features=["RUL"])


raw_transformer = Transformer(
    transformerX=pipe_raw,
    transformerY=y_pipe
)


# Sliding window iterators

In [ ]:
window = 500
train_iterator = WindowedDatasetIterator(
    transformed_train_dataset,
    window,
    step=1,
    
    start_index=RelativeToEnd(3000),
    shuffler=AllShuffled(),

)

train_not_shuffled_iterator = WindowedDatasetIterator(
    transformed_train_dataset,
    window,
    step=5,
    
    start_index=RelativeToEnd(3000),
)

val_iterator = WindowedDatasetIterator(
    transformed_val_dataset,
    window,
    step=15,
    
    shuffler=NotShuffled(),
    start_index=RelativeToEnd(3000),
    
)

# Model

In [ ]:




train_set, val_set = train_test_split(dataset, train_size=0.8, shuffle=True)

transformed_train_dataset = train_set.map(transformer)
transformed_val_dataset = val_set.map(transformer)

